# Import Libraries

In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import nltk
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Functions

In [ ]:
def clean_up(s):
  """
  Cleans up numbers, URLs, and special characters from a string.

  Args:
      s: The string to be cleaned up.

  Returns:
      A string that has been cleaned up.
  """
  
  s = s.lower()

  # Replace urls by ' '
  s = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', ' ' , s) 

  # Replace everything non-alpahnumeric by ' '
  s = re.sub(r'\W+',' ',s) 

  # Replace one or more digits by  ' '
  s = re.sub(r'\d+',' ',s) 

  # Replace one or more whitespaces by  ' '
  #s = re.sub(r'\s+',' ',s) 
  
  return s

In [ ]:
def tokenize(s):

  """
  Tokenize a string.

  Args:
      s: String to be tokenized.

  Returns:
      A list of words as the result of tokenization.
  """

  s = s.split()
  return s


In [ ]:
def stem_and_lemmatize(l):
  """
  Perform stemming and lemmatization on a list of words.

  Args:
      l: A list of strings.

  Returns:
      A list of strings after being stemmed and lemmatized.
  """
  #import nltk
  #nltk.download('omw-1.4')
  #nltk.download('wordnet')

  #from nltk.stem import WordNetLemmatizer

  lem = WordNetLemmatizer()

  l = [lem.lemmatize(word) for word in l]
  return l

In [ ]:
def remove_stopwords(l):

  """
  Remove English stopwords from a list of strings.

  Args:
      l: A list of strings.

  Returns:
      A list of strings after stop words are removed.
  """

  # nltk.download('stopwords')
  # from nltk.corpus import stopwords

  stop_words = list(stopwords.words('english'))
  for i in range(len(stop_words)):
    stop_words[i] = re.sub(r"\s*'\s*\w*","",stop_words[i])

  l= [w for w in l if (w not in stop_words) and (len(w)>1)]

  return l

# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [ ]:
# your code here

In [ ]:
tweets = pd.read_csv('/content/drive/MyDrive/ironhack-lab_8.3/training.1600000.processed.noemoticon.csv', encoding = 'latin-1')
tweets

# UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 232719-232720: invalid continuation byte

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
tweets.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
tweets.head()

,target,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1599999 non-null  int64 
 1   id      1599999 non-null  int64 
 2   date    1599999 non-null  object
 3   flag    1599999 non-null  object
 4   user    1599999 non-null  object
 5   text    1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


Sub sample the data frame by target classes.  
Another way would be to choose the most recent tweets.  

In [ ]:
tweets.target.value_counts(dropna=False)

4    800000
0    799999
Name: target, dtype: int64

In [ ]:
subtweets0 = tweets[tweets.target == 0]
subtweets0 = subtweets0.sample(10000, axis=0, random_state=0)
subtweets0.target.value_counts()

0    10000
Name: target, dtype: int64

In [ ]:
subtweets4 = tweets[tweets.target == 4]
subtweets4 = subtweets4.sample(10000, axis=0, random_state=0)
subtweets4.target.value_counts()

4    10000
Name: target, dtype: int64

In [ ]:
st = pd.concat([subtweets0,subtweets4], axis=0).sort_index()
st

,target,id,date,flag,user,text
56,0,1467822389,Mon Apr 06 22:22:47 PDT 2009,NO_QUERY,HybridMink,I have a sad feeling that Dallas is not going ...
284,0,1467881897,Mon Apr 06 22:38:30 PDT 2009,NO_QUERY,drtaru,@zaydia but i cant figure out how to get there...
323,0,1467894786,Mon Apr 06 22:41:55 PDT 2009,NO_QUERY,otherbella,@ridley1013 I agree. The shapeshifting is a co...
408,0,1467916595,Mon Apr 06 22:47:58 PDT 2009,NO_QUERY,LaTtEX,@robcthegeek Once upon a time Hundred Islands ...
513,0,1467943966,Mon Apr 06 22:55:48 PDT 2009,NO_QUERY,Lora_June,@Miss_Sil no i was half asleep and turned off ...
...,...,...,...,...,...,...
1599674,4,2193503297,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,peachtweet,"yesterday,my house smelt like that really cool..."
1599733,4,2193526487,Tue Jun 16 08:34:40 PDT 2009,NO_QUERY,thenewpsies,@shelovesshoes hehehe don't keep up with new b...
1599747,4,2193527300,Tue Jun 16 08:34:44 PDT 2009,NO_QUERY,miss_extreme,WE GET TO KEEP OUR HOUSE!
1599823,4,2193551468,Tue Jun 16 08:36:42 PDT 2009,NO_QUERY,slimgoodies,@hillsmarket Sweet. Thanks for posting that. ...


Check the dates

In [ ]:
tweets.date = pd.to_datetime(tweets.date)

/usr/local/lib/python3.7/dist-packages/dateutil/parser/_parser.py:1212: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [ ]:
tweets.head(10)

,target,id,date,flag,user,text
0,0,1467810672,2009-04-06 22:19:49,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,2009-04-06 22:19:53,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,2009-04-06 22:19:57,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,2009-04-06 22:19:57,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,2009-04-06 22:20:00,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
5,0,1467811592,2009-04-06 22:20:03,NO_QUERY,mybirch,Need a hug
6,0,1467811594,2009-04-06 22:20:03,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
7,0,1467811795,2009-04-06 22:20:05,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
8,0,1467812025,2009-04-06 22:20:09,NO_QUERY,mimismo,@twittera que me muera ?
9,0,1467812416,2009-04-06 22:20:16,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing


In [ ]:
tweets.shape

(1599999, 6)

In [ ]:
tweets.sort_values('date', ascending = False).iloc[:20000,0].value_counts()

0    20000
Name: target, dtype: int64

It seems the last 20k tweets are negative...

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [ ]:
# your code here

In [ ]:
st['text_processed'] = list(map(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))),st.text))
st

,target,id,date,flag,user,text,text_processed
56,0,1467822389,Mon Apr 06 22:22:47 PDT 2009,NO_QUERY,HybridMink,I have a sad feeling that Dallas is not going ...,"[sad, feeling, dallas, going, show, gotta, say..."
284,0,1467881897,Mon Apr 06 22:38:30 PDT 2009,NO_QUERY,drtaru,@zaydia but i cant figure out how to get there...,"[zaydia, cant, figure, get, back, pay, hotel, ..."
323,0,1467894786,Mon Apr 06 22:41:55 PDT 2009,NO_QUERY,otherbella,@ridley1013 I agree. The shapeshifting is a co...,"[ridley, agree, shapeshifting, copout, wa, exc..."
408,0,1467916595,Mon Apr 06 22:47:58 PDT 2009,NO_QUERY,LaTtEX,@robcthegeek Once upon a time Hundred Islands ...,"[robcthegeek, upon, time, hundred, island, wa,..."
513,0,1467943966,Mon Apr 06 22:55:48 PDT 2009,NO_QUERY,Lora_June,@Miss_Sil no i was half asleep and turned off ...,"[miss_sil, wa, half, asleep, turned, laptop]"
...,...,...,...,...,...,...,...
1599674,4,2193503297,Tue Jun 16 08:32:47 PDT 2009,NO_QUERY,peachtweet,"yesterday,my house smelt like that really cool...","[yesterday, house, smelt, like, really, cool, ..."
1599733,4,2193526487,Tue Jun 16 08:34:40 PDT 2009,NO_QUERY,thenewpsies,@shelovesshoes hehehe don't keep up with new b...,"[shelovesshoes, hehehe, keep, new, book, seee,..."
1599747,4,2193527300,Tue Jun 16 08:34:44 PDT 2009,NO_QUERY,miss_extreme,WE GET TO KEEP OUR HOUSE!,"[get, keep, house]"
1599823,4,2193551468,Tue Jun 16 08:36:42 PDT 2009,NO_QUERY,slimgoodies,@hillsmarket Sweet. Thanks for posting that. ...,"[hillsmarket, sweet, thanks, posting, letting,..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [ ]:
# your code here

In [ ]:
corpus = []
for x in st.text_processed:
  corpus += x



In [ ]:
corpus

In [ ]:
len(corpus)

150458

In [ ]:
wordfreq = {} # A dictionary to hold as keys every single word, and the value is how many times appears the word.

for word in corpus:
  if word not in wordfreq.keys():
    wordfreq[word] = 1 # We initialize the corresponding counter
  else:
    wordfreq[word] += 1 # We increase the corresponding counter

wordfreq

In [ ]:
len(wordfreq.keys())

26802

In [ ]:
wordfreq = sorted(wordfreq.items(), key = lambda x: x[1], reverse = True)

wordfreq

In [ ]:
important_words = wordfreq[:5000]
important_words

[('day', 1366),
 ('wa', 1320),
 ('good', 1177),
 ('get', 1113),
 ('like', 1034),
 ('go', 1022),
 ('quot', 954),
 ('love', 890),
 ('work', 866),
 ('time', 828),
 ('going', 820),
 ('today', 812),
 ('got', 794),
 ('back', 731),
 ('lol', 708),
 ('one', 699),
 ('really', 676),
 ('know', 673),
 ('want', 646),
 ('im', 642),
 ('see', 601),
 ('amp', 600),
 ('new', 550),
 ('night', 544),
 ('still', 541),
 ('think', 533),
 ('oh', 528),
 ('home', 521),
 ('well', 513),
 ('need', 504),
 ('ha', 497),
 ('much', 494),
 ('thanks', 485),
 ('miss', 472),
 ('twitter', 454),
 ('make', 450),
 ('morning', 444),
 ('feel', 438),
 ('last', 433),
 ('tomorrow', 431),
 ('hope', 422),
 ('haha', 405),
 ('great', 386),
 ('fun', 381),
 ('wish', 373),
 ('right', 367),
 ('would', 365),
 ('way', 365),
 ('come', 360),
 ('happy', 358),
 ('sad', 350),
 ('bad', 344),
 ('week', 341),
 ('friend', 331),
 ('thing', 322),
 ('tonight', 321),
 ('sleep', 319),
 ('sorry', 315),
 ('nice', 307),
 ('look', 307),
 ('hate', 302),
 ('better

Try with FreqDist

In [ ]:
corpus2 = nltk.FreqDist(corpus)
corpus2

In [ ]:
type(corpus2)

nltk.probability.FreqDist

In [ ]:
important_2 = sorted(corpus2.items(), key = lambda x: x[1], reverse = True)
important_2

[('day', 1366),
 ('wa', 1320),
 ('good', 1177),
 ('get', 1113),
 ('like', 1034),
 ('go', 1022),
 ('quot', 954),
 ('love', 890),
 ('work', 866),
 ('time', 828),
 ('going', 820),
 ('today', 812),
 ('got', 794),
 ('back', 731),
 ('lol', 708),
 ('one', 699),
 ('really', 676),
 ('know', 673),
 ('want', 646),
 ('im', 642),
 ('see', 601),
 ('amp', 600),
 ('new', 550),
 ('night', 544),
 ('still', 541),
 ('think', 533),
 ('oh', 528),
 ('home', 521),
 ('well', 513),
 ('need', 504),
 ('ha', 497),
 ('much', 494),
 ('thanks', 485),
 ('miss', 472),
 ('twitter', 454),
 ('make', 450),
 ('morning', 444),
 ('feel', 438),
 ('last', 433),
 ('tomorrow', 431),
 ('hope', 422),
 ('haha', 405),
 ('great', 386),
 ('fun', 381),
 ('wish', 373),
 ('right', 367),
 ('would', 365),
 ('way', 365),
 ('come', 360),
 ('happy', 358),
 ('sad', 350),
 ('bad', 344),
 ('week', 341),
 ('friend', 331),
 ('thing', 322),
 ('tonight', 321),
 ('sleep', 319),
 ('sorry', 315),
 ('nice', 307),
 ('look', 307),
 ('hate', 302),
 ('better

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [ ]:
# your code here

In [ ]:
word_features = [x[0] for x in important_words]
word_features

In [ ]:
feature_set = []

for idx,token in enumerate(st.text_processed):
  # Initializing an empty dictionary of boolean features
  feature = {word:False for word in word_features}

  # Filling the dictionary with the boolean values
  for word in word_features:
    if word in token:
      feature[word] = True

  # Add sentiment
  if st.iloc[idx,0] == 0:
    feature = (feature, False)
  else:
    feature = (feature, True)

  feature_set.append(feature)


In [ ]:
#list(st.columns).index('target')

0

In [ ]:
feature_set

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [ ]:
# your code here

In [68]:
# Split train and test
len(feature_set)*0.8

16000.0

In [69]:
train_set, test_set = feature_set[:16000], feature_set[16000:]

In [71]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [72]:
classifier.show_most_informative_features()

Most Informative Features
                     sad = True            False : True   =     44.1 : 1.0
                headache = True            False : True   =     17.6 : 1.0
                    poor = True            False : True   =     17.4 : 1.0
            followfriday = True             True : False  =     15.5 : 1.0
                     ugh = True            False : True   =     14.7 : 1.0
                horrible = True            False : True   =     13.8 : 1.0
                   worse = True            False : True   =     13.4 : 1.0
                 missing = True            False : True   =     10.6 : 1.0
                   thank = True             True : False  =      9.6 : 1.0
                    doll = True             True : False  =      9.4 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here

In [73]:
nltk.classify.accuracy(classifier, test_set)

0.59025

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here